In [1]:
import pandas
import datetime
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
ab_loc = pandas.read_csv('2016-02-16/listings.csv.gz', compression='gzip', usecols=['host_id','city','state','price','number_of_reviews'])

In [3]:
print(len(ab_loc))
ab_loc.dropna(inplace=True)
print(len(ab_loc))

337213
336360


In [4]:
ab_loc['city'] = ab_loc['city'].apply(lambda x: x.strip().lower())
ab_loc['state'] = ab_loc['state'].apply(lambda x: x.strip().lower())
ab_loc['citystate'] = ab_loc['city'] + '_' + ab_loc['state']
ab_loc['price'] = ab_loc['price'].apply(lambda x: float(x.split('$')[1].replace(',','')) )

In [5]:
ab_refver = ab_loc[ab_loc['number_of_reviews']>=1]
ab_refver.head()

,host_id,city,state,price,number_of_reviews,citystate
0,5803,washington,dc,55,13,washington_dc
2,36016,hollidaysburg,pa,95,18,hollidaysburg_pa
3,38067,kihei,hi,100,4,kihei_hi
6,49162,paauilo,hi,120,2,paauilo_hi
7,55469,key largo,fl,200,12,key largo_fl


In [6]:
ab_lookup = ab_refver[['citystate','price']]

In [7]:
ab_lookup = ab_lookup.reset_index(drop=True)

# Q: at what max_price does an AB city have n hosts?

In [8]:
price_sort = ab_refver[ab_refver['citystate'] == 'san francisco_ca'].sort('price')

price_sort.head()

,host_id,city,state,price,number_of_reviews,citystate
201242,1682188,san francisco,ca,30,11,san francisco_ca
32566,1682188,san francisco,ca,30,13,san francisco_ca
159221,1682188,san francisco,ca,30,15,san francisco_ca
327785,1682188,san francisco,ca,30,16,san francisco_ca
327782,1682188,san francisco,ca,30,11,san francisco_ca


In [9]:
cs_df = pandas.read_csv('couchsurfing/clean_hosts.csv',usecols=['people_id','member_since','city','state','status','verified_status','references_int'])

def get_year(s):
    try:
        return datetime.datetime(int(s.split("Member since ")[1]), 1, 1)
    except:
        return None
    
cs_df['member_since'] = cs_df['member_since'].apply(get_year)
cs_defhosts = cs_df[(cs_df['status'] == 'Accepting Guests')]
cs_defhosts['any_verification'] = cs_defhosts['verified_status'].apply(lambda x: int(x != 'Not Verified'))
cs_defhosts['any_references'] = cs_defhosts['references_int'].apply(lambda x: int(x != 0))
cs_defhosts['any_refver'] = cs_defhosts[['any_references','any_verification']].apply(lambda row: row[0] or row[1], axis=1)

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

In [10]:
cs_refver = cs_defhosts[cs_defhosts['any_refver']==True]

In [11]:
len(cs_refver)

54258

In [12]:
state_dict = dict( (('Alabama', 'AL'),
       ('Alaska', 'AK'),
       ('Arizona', 'AZ'),
       ('Arkansas', 'AR'),
       ('California', 'CA'),
       ('Colorado', 'CO'),
       ('Connecticut', 'CT'),
       ('Delaware', 'DE'),
       ('District of Columbia', 'DC'),
       ('Florida', 'FL'),
       ('Georgia', 'GA'),
       ('Hawaii', 'HI'),
       ('Idaho', 'ID'),
       ('Illinois', 'IL'),
       ('Indiana', 'IN'),
       ('Iowa', 'IA'),
       ('Kansas', 'KS'),
       ('Kentucky', 'KY'),
       ('Louisiana', 'LA'),
       ('Maine', 'ME'),
       ('Maryland', 'MD'),
       ('Massachusetts', 'MA'),
       ('Michigan', 'MI'),
       ('Minnesota', 'MN'),
       ('Mississippi', 'MS'),
       ('Missouri', 'MO'),
       ('Montana', 'MT'),
       ('Nebraska', 'NE'),
       ('Nevada', 'NV'),
       ('New Hampshire', 'NH'),
       ('New Jersey', 'NJ'),
       ('New Mexico', 'NM'),
       ('New York', 'NY'),
       ('North Carolina', 'NC'),
       ('North Dakota', 'ND'),
       ('Ohio', 'OH'),
       ('Oklahoma', 'OK'),
       ('Oregon', 'OR'),
       ('Pennsylvania', 'PA'),
       ('Rhode Island', 'RI'),
       ('South Carolina', 'SC'),
       ('South Dakota', 'SD'),
       ('Tennessee', 'TN'),
       ('Texas', 'TX'),
       ('Utah', 'UT'),
       ('Vermont', 'VT'),
       ('Virginia', 'VA'),
       ('Washington', 'WA'),
       ('West Virginia', 'WV'),
       ('Wisconsin', 'WI'),
       ('Wyoming', 'WY')) )

def state_lookup(s):

    if len(s) ==2:
        return s.lower()
    else:
        try:
            return state_dict[s].lower()
        except KeyError:
            print(s)
            return float('nan')
        
cs_refver['state'] = cs_refver['state'].map(state_lookup)
cs_refver.dropna(inplace=True)
cs_refver['city'] = cs_refver['city'].apply(lambda x: x.strip().lower())
cs_refver.dropna(inplace=True)
cs_refver['state'] = cs_refver['state'].apply(lambda x: x.strip().lower())
cs_refver['citystate'] = cs_refver['city'] + '_' + cs_refver['state']

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.4/dist-package

Birmingham
Ames


/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
cs_citystate = cs_refver.groupby(by='citystate').agg({'people_id':len})

In [14]:
cs_citystate = cs_citystate.reset_index()
cs_citystate.columns = ['citystate','num_hosts']

In [15]:
def price_for_n_hosts(citystate, n):
    ab_citystate = ab_lookup[ab_lookup['citystate']==citystate].sort('price')
    #print(citystate, n)
    #print(ab_citystate)
    total_ab = len(ab_citystate)
    if total_ab==0:
        return False
    if n > total_ab:
        return max(ab_citystate['price'])
    else:
        min_price = ab_citystate.iloc[n-1]['price']#n-1 because if cs has one person, we want the price of the zeroeth cheapest ab
        return min_price

In [16]:
cs_citystate['ab_min_price'] = cs_citystate.apply(lambda x: price_for_n_hosts(x['citystate'], x['num_hosts']), axis=1)

In [17]:
cs_citystate.head(20)

,citystate,num_hosts,ab_min_price
0,abbeville_la,1,110
1,abbeville_sc,1,False
2,aberdeen_md,1,False
3,aberdeen_nc,3,75
4,aberdeen_oh,1,False
5,aberdeen_sd,13,75
6,aberdeen_wa,2,46
7,abilene_tx,14,350
8,abingdon_va,4,50
9,abington_ma,2,False


In [18]:
cs_citystate_abequiv  = cs_citystate[cs_citystate['ab_min_price']!= False]
cs_citystate_noequiv  = cs_citystate[cs_citystate['ab_min_price']== False]

In [45]:
cs_citystate_abequiv[cs_citystate_abequiv['citystate']=='west hollywood_ca']

,citystate,num_hosts,ab_min_price
6392,west hollywood_ca,40,65


In [20]:
len(cs_citystate_noequiv)/len(cs_citystate)

0.3116126139590495

In [21]:
cs_citystate_noequiv['num_hosts'].sum() / cs_citystate['num_hosts'].sum()

0.055055722166685181

In [37]:
cs_citystate_abequiv.describe()

,num_hosts
count,4606.000000
mean,11.082067
std,56.626392
min,1.000000
25%,1.000000
50%,2.000000
75%,5.000000
max,1456.000000


$11

In [23]:
%load_ext rpy2.ipython

In [24]:
%Rpush cs_citystate_abequiv

In [25]:
%%R
library(ggplot2)

p = ggplot(cs_citystate_abequiv, aes(x=ab_min_price)) + geom_histogram()
print(p)


Error: StatBin requires a continuous x variable the x variable is discrete. Perhaps you want stat="count"?


/usr/local/lib/python3.4/dist-packages/rpy2/robjects/functions.py:106: UserWarning: Error: StatBin requires a continuous x variable the x variable is discrete. Perhaps you want stat="count"?

  res = super(Function, self).__call__(*new_args, **new_kwargs)
